# 3. MC 문제

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Pseudo-Lab/Tutorial-Book/blob/master/book/chapters/NLP/Ch3-MC-Question.ipynb)

이전 장에서는 실습에 사용할 데이터셋을 다운로드하고 시각화하며 전처리를 진행해보았습니다. 이번 장에서는 해당 데이터셋을 이용하여 MC 문제를 생성하는 모델을 실습해보도록 하겠습니다.
    
    
3.1절에서는 문제를 생성하는 데에 사용되는 GPT-2와 BERT 모델을 소개하고, 3.2절에서는 데이터를 불러와서 다시 한 번 살펴봅니다. 이어 3.3절에서는 T5 모델을 이용하여 Text 데이터를 요약하는 작업을 진행하며, 마지막으로 3.4절에서는 텍스트 속의 문장들을 필터링하고 GPT-2와 BERT를 이용하여 MC 문제를 생성해보도록 하겠습니다.

MC 문제를 생성하는 과정은 크게 5단계로 설명될 수 있습니다. 가장 먼저 MC 문제를 생성하기 위한 지문이 되는 텍스트로부터 주요 문장들을 추출하여 요약합니다. 두번째, 추출된 주요 문장들을 유사 어휘와 어구를 활용하여 변환하며(pharaphrasing) 세번째, 이 변환된 문장들을 파싱합니다. 네번째로 이 파싱된 문장들과 GPT-2 모델을 이용해 거짓 문장을 생성하고, 마지막으로 유사도를 평가해서 정답 문장과 가장 유사하지 않은 문장들을 오답 선택지로 활용합니다. 이 과정은 3.2절부터 순서대로 확인할 수 있습니다.

## 3.1 GPT2 & BERT

### 3.1.1 GPT2

GPT-2는 OpenAI에서 개발한 GPT-N 시리즈의 2번째 자연어처리 모델입니다. GPT-2는 8백만 개의 웹페이지 데이터셋과 15억 개의 파라미터로부터 학습된 트랜스포머 기반 자연어처리 모델이며 이전의 단어들을 포함하는 텍스트 다음에 올 단어를 예측하는 것을 목적으로 짜여진 모델입니다. 우리는 MC 문제 생성 태스크에서 거짓 문장을 생성하는 데에 GPT-2를 활용하게 됩니다.

### 3.1.2 BERT

BERT는 Bidirectional Encoder Representations from Transformers 의 약어로 구글에서 2018년 개발한 자연어처리 모델입니다. BERT는 Transformer를 기반으로 Sentence Embedding 혹은 Contextual Word Embedding을 구하는 네트워크로, 문장을 토큰 단위로 쪼개서 네트워크에 넣으면 전체 문장에 대한 vector와 문장 안의 단어 각각에 대응되는 vector를 출력합니다. 이를 기반으로 Text Classification 등의 Task를 학습하여 수행할 수 있습니다. 

![](https://github.com/Pseudo-Lab/Tutorial-Book/blob/master/book/pics/NLP-ch1img02.png?raw=true)

 - 그림 3.1 전반적인 BERT의 pre-training 과정과 fine-tuning 과정 (출처: BERT: Pre-training of Deep Bidirectional Transformers for
Language Understanding)

NLP Task 성능 평가와 관련하여 다양한 NLP Task들의 성능을 바탕으로 모델들의 순위를 매기는 GLUE Benchmark(General Language Understatnding Evaluation Benchmark)라는 collection이 있는데, BERT는 여기에서 OpenAI GPT 등의 다른 모델들을 큰 차이로 앞서며 그 당시 최고의 성능을 보여 주었습니다.

## 3.2 데이터셋 다운로드

2장에서 나온 코드를 활용하여 데이터셋을 불러오도록 하겠습니다. 데이터셋은 올바른 문법으로 수정이 된 에세이 텍스트 데이터입니다. 이 데이터는 MC 문제를 만들 지문으로 사용됩니다. 데이터를 읽어 오기 위해서 pickle 패키지를 이용합니다.

In [1]:
import pickle

!git clone https://github.com/Pseudo-Lab/Tutorial-Book-Utils
!python Tutorial-Book-Utils/PL_data_loader.py --data NLP-QG
file_name = "CoNLL+BEA_corrected_essays.pkl"
open_file = open(file_name, "rb")
data = pickle.load(open_file)
open_file.close()

Cloning into 'Tutorial-Book-Utils'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 30 (delta 9), reused 18 (delta 5), pack-reused 0
Unpacking objects: 100% (30/30), done.
CoNLL+BEA_corrected_essays.pkl is done!


In [2]:
!pip install -q benepar
!pip install -q sentence_transformers

import requests
import json
import benepar
import string
import nltk
from nltk import tokenize
from nltk.tokenize import sent_tokenize
from string import punctuation
import re
from random import shuffle
import spacy
import warnings
import torch
import pandas as pd
import numpy as np
import scipy
torch.manual_seed(42)

warnings.filterwarnings(action='ignore')
nlp = spacy.load('en')

nltk.download('punkt')

benepar.download('benepar_en3')
benepar_parser = benepar.Parser("benepar_en3")

     |████████████████████████████████| 3.3 MB 20.3 MB/s 
     |████████████████████████████████| 2.8 MB 10.7 MB/s 
     |████████████████████████████████| 1.2 MB 34.3 MB/s 
     |████████████████████████████████| 895 kB 52.9 MB/s 
     |████████████████████████████████| 636 kB 53.8 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 85 kB 3.0 MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package benepar_en3 to /root/nltk_data...
[nltk_data]   Unzipping models/benepar_en3.zip.


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
def preprocess(sentences):
    
    output = []
    for sent in sentences:
        single_quotes_present = len(re.findall(r"['][\w\s.:;,!?\\-]+[']",sent))>0
        double_quotes_present = len(re.findall(r'["][\w\s.:;,!?\\-]+["]',sent))>0
        question_present = "?" in sent
        if single_quotes_present or double_quotes_present or question_present :
            continue
        else:
            output.append(sent.strip(punctuation))
    return output


In [5]:
def get_flattened(t):
    sent_str_final = None
    if t is not None:
        sent_str = [" ".join(x.leaves()) for x in list(t)]
        sent_str_final = [" ".join(sent_str)]
        sent_str_final = sent_str_final[0]
    return sent_str_final
    
def get_termination_portion(main_string,sub_string):
    combined_sub_string = sub_string.replace(" ","")
    main_string_list = main_string.split()
    last_index = len(main_string_list)
    for i in range(last_index):
        check_string_list = main_string_list[i:]
        check_string = "".join(check_string_list)
        check_string = check_string.replace(" ","")
        if check_string == combined_sub_string:
            return " ".join(main_string_list[:i])
                     
    return None
    
def get_right_most_VP_or_NP(parse_tree,last_NP = None,last_VP = None):
    if len(parse_tree.leaves()) == 1:
        return get_flattened(last_NP),get_flattened(last_VP)
    last_subtree = parse_tree[-1]
    if last_subtree.label() == "NP":
        last_NP = last_subtree
    elif last_subtree.label() == "VP":
        last_VP = last_subtree
    
    return get_right_most_VP_or_NP(last_subtree,last_NP,last_VP)


def get_sentence_completions(key_sentences):
    sentence_completion_dict = {}
    for individual_sentence in key_sentences:
        sentence = individual_sentence.rstrip('?:!.,;')
        tree = benepar_parser.parse(sentence)
        last_nounphrase, last_verbphrase =  get_right_most_VP_or_NP(tree)
        phrases= []
        if last_verbphrase is not None:
            verbphrase_string = get_termination_portion(sentence,last_verbphrase)
            if verbphrase_string is not None:
                phrases.append(verbphrase_string)
        if last_nounphrase is not None:
            nounphrase_string = get_termination_portion(sentence,last_nounphrase)
            if nounphrase_string is not None:
                phrases.append(nounphrase_string)
    
        longest_phrase =  sorted(phrases, key=len, reverse=True)
        if len(longest_phrase) == 2:
            first_sent_len = len(longest_phrase[0].split())
            second_sentence_len = len(longest_phrase[1].split())
            if (first_sent_len - second_sentence_len) > 4:
                del longest_phrase[1]
                
        if len(longest_phrase)>0:
            sentence_completion_dict[sentence]=longest_phrase

    return sentence_completion_dict

In [6]:
def sort_by_similarity(original_sentence, generated_sentences_list):

    sentence_embeddings = bert_model.encode(generated_sentences_list)

    queries = [original_sentence]

    query_embeddings = bert_model.encode(queries)

    number_top_matches = len(generated_sentences_list)

    dissimilar_sentences = []


    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])

        for idx, distance in reversed(results[0:number_top_matches]):
            score = 1-distance
            # print(score)
            if score < 0.99:
                dissimilar_sentences.append(generated_sentences_list[idx].strip())
           
    sorted_dissimilar_sentences = sorted(dissimilar_sentences, key=len)
    
    return sorted_dissimilar_sentences[:2]
    

def generate_sentences(partial_sentence,full_sentence):
    input_ids = gpt2_tokenizer.encode(partial_sentence, return_tensors='pt') # use tokenizer to encode
    input_ids = input_ids.to(device)
    maximum_length = len(partial_sentence.split())+80 

    sample_outputs = gpt2_model.generate( 
        input_ids, 
        do_sample=True, 
        max_length=maximum_length, 
        top_p=0.90, 
        top_k=50,   
        repetition_penalty  = 10.0,
        num_return_sequences=5
    )
    generated_sentences=[]
    for i, sample_output in enumerate(sample_outputs):
        decoded_sentences = gpt2_tokenizer.decode(sample_output, skip_special_tokens=True)
        decoded_sentences_list = tokenize.sent_tokenize(decoded_sentences)
        generated_sentences.append(decoded_sentences_list[0]) # takes the first sentence 
        
    top_3_sentences = sort_by_similarity(full_sentence, generated_sentences)
    
    return top_3_sentences


In [7]:
## load models
from transformers import AutoModelWithLMHead, AutoTokenizer, AutoModelForSeq2SeqLM
import torch

summarize_tokenizer = AutoTokenizer.from_pretrained("t5-small")
paraphrase_tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws") 
gpt2_tokenizer = AutoTokenizer.from_pretrained("gpt2")

summarize_model = AutoModelWithLMHead.from_pretrained("t5-small")
paraphrase_model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws")
# add the EOS token as PAD token to avoid warnings
gpt2_model = AutoModelWithLMHead.from_pretrained("gpt2", pad_token_id=gpt2_tokenizer.eos_token_id) 

summarize_model.to(device)
paraphrase_model.to(device)
gpt2_model.to(device)


from sentence_transformers import SentenceTransformer
bert_model = SentenceTransformer('bert-base-nli-mean-tokens')

bert_model.to(device)

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [8]:
df_TFQuestions = pd.DataFrame({'id': np.zeros(20),
                               'passage': np.zeros(20),
                               'distractor_1': np.zeros(20),
                               'distractor_2': np.zeros(20),
                               'distractor_3': np.zeros(20),
                               'distractor_4': np.zeros(20)})

A: raw  
A': paraphrased  
A_False: false


- distractor_1: A'
- distractor_2: A'_False
- distractor_3: B'_False
- distractor_4: C'_False

In [9]:
## main.py

import random
random.seed(42)

passage_id_list = [163,
 28,
 62,
 57,
 35,
 26,
 22,
 151,
 108,
 55,
 59,
 129,
 167,
 143,
 50,
 161,
 107,
 56,
 114,
 71]

for id_idx in range(20):

    # select passage for question generation 
    passage_id = passage_id_list[id_idx]

    passage = data[passage_id]

    # summarize
    inputs = summarize_tokenizer.encode("summarize: " + passage, return_tensors="pt", max_length=512)

    inputs = inputs.to(device)

    outputs = summarize_model.generate(inputs, max_length=300, min_length=100, length_penalty=2.0, num_beams=4, early_stopping=True)

    extractedSentences = summarize_tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    tokenized_sentences = nltk.tokenize.sent_tokenize(extractedSentences)

    filter_quotes_and_questions = preprocess(tokenized_sentences)

    # paraphrase

    paraphrased_sentences = []

    for summary_idx in range(len(filter_quotes_and_questions)):

        sentence = filter_quotes_and_questions[summary_idx]

        inputs = "paraphrase: " + sentence + " </s>"

        encoding = paraphrase_tokenizer.encode_plus(inputs, pad_to_max_length=True, return_tensors="pt")
        input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

        outputs = paraphrase_model.generate(
            input_ids=input_ids, attention_mask=attention_masks,
            max_length=256,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            early_stopping=True,
            num_return_sequences=3
            )

        paraphrased_sentences.append(paraphrase_tokenizer.decode(outputs[0], skip_special_tokens=True,clean_up_tokenization_spaces=True))

        if len(paraphrased_sentences) == 3:
            print('3 filled')
            break

        if (summary_idx == (len(filter_quotes_and_questions) - 1)) & (len(paraphrased_sentences) < 3): # 마지막인데 채워지지 않았을 경우 존재하는 paraphrased sentence 반복해서 false 문장 생성
            print(summary_idx)
            print('hit')
            for paraphrase_idx in range(1, 3):
                paraphrased_sentences.append(paraphrase_tokenizer.decode(outputs[paraphrase_idx], skip_special_tokens=True,clean_up_tokenization_spaces=True))


    sent_completion_dict = get_sentence_completions(paraphrased_sentences)

    df_TFQuestions.loc[id_idx, 'id'] = passage_id
    df_TFQuestions.loc[id_idx, 'passage'] = passage
    df_TFQuestions.loc[id_idx, 'distractor_1'] = list(sent_completion_dict.keys())[0]

    distractor_cnt = 2

    for key_sentence in sent_completion_dict:

        if distractor_cnt == 5:
            break

        partial_sentences = sent_completion_dict[key_sentence]
        false_sentences =[]
        # df_TFQuestions.loc[0, 'id'] = 
        # print_string = "**%s) True Sentence (from the story) :**"%(str(index))
        # printmd(print_string)
        # print ("  ",key_sentence)
        
        false_sents = []
        for partial_sent in partial_sentences:
            
            for repeat in range(10):
                false_sents = generate_sentences(partial_sent, key_sentence)
                if false_sents != []:
                    break
                    
            false_sentences.extend(false_sents)
        
        df_TFQuestions.loc[id_idx, f'distractor_{distractor_cnt}'] = false_sentences[0]
        
        distractor_cnt += 1
    
    print(id_idx, 'complete')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


3 filled
0 complete
3 filled
1 complete
3 filled
2 complete
3 filled
3 complete
3 filled
4 complete
1
hit
5 complete
3 filled
6 complete
3 filled
7 complete
3 filled
8 complete
3 filled
9 complete
3 filled
10 complete
3 filled
11 complete
3 filled
12 complete
3 filled
13 complete
3 filled
14 complete
3 filled
15 complete
3 filled
16 complete
3 filled
17 complete
1
hit
18 complete
3 filled
19 complete


In [10]:
df_TFQuestions

,id,passage,distractor_1,distractor_2,distractor_3,distractor_4
0,163.0,The waters of the culinary seas had been calm ...,The microwave is the source of life most of us...,The microwave is the source of life most of us...,It uses radiation to excite water particles in...,"The microwave cuts the cooking time half, maki..."
1,28.0,The world is increasingly becoming flat with a...,Social networks sites provide us with many con...,Social networks sites provide us with the tool...,The news can be found by scanning the webpage ...,A piece of research shows that people unconsci...
2,62.0,The best places for y...,The report looks at the best places to visit f...,The report looks at the best places to visit f...,It is based on a survey of young people from a...,It is based on my own opinion as a permanent r...
3,57.0,Puerquitour: A great experience for your mouth...,The place is Tacos La Chule and there are gour...,The place is Tacos La Chule and there are thre...,The name of the place is Tacos La Chule and th...,"The place is sooooo cute, the decoration and e..."
4,35.0,"Nowadays, social media sites are commonly used...",80% of people use social media sites to connec...,80% of people use social media sites to connec...,"They consist of the function of a single loop,...",but there are also disadvantages which occur t...
5,26.0,"Interpersonal skills, like any other skills re...",The burgeoning of social media has its benefit...,The burgeoning of social media has its benefit...,It is a good practice not to use social media ...,It is a good practice to not use such features.
6,22.0,"Nowadays, with the advancement of technology, ...",a known genetic risk should not be obligated t...,a known genetic risk should not be obligated t...,The government should set the law to protect t...,"However, the carrier of a known genetic risk m..."
7,151.0,In this century there have been many technolog...,Television brought other worlds right into the...,Television brought other worlds right into the...,Television has the power to bring wars to the ...,"In the minds of most Americans, television is ..."
8,108.0,"I met a friend about one week ago, and he aske...",It's about a teen couple who are dying of canc...,It's about a teen couple who are dying of canc...,It's about a teen couple who are dying from ca...,Now I have an awful feeling about what I am do...
9,55.0,"Dear Sir or Madam,\nI am writing to apply for ...",The Summer Camp Counsellor is currently advert...,The Summer Camp Counsellor is currently advert...,"At the moment, I have finished the second year...","XYZ, who was my boss at the Scout camp, I am l..."


In [11]:
df_TFQuestions.to_csv('TFQuestions.csv', index=False)

## 3.3 참고문헌



1. [Practical AI : Automatically Generate True or False questions from any content with OpenAI GPT2, Sentence BERT and Berkley Constituency parser](https://medium.com/swlh/practical-ai-automatically-generate-true-or-false-questions-from-any-content-with-openai-gpt2-9081ffe4d4c9)

2. https://openai.com/blog/better-language-models/

3. https://github.com/openai/gpt-2

4. https://arxiv.org/abs/1810.04805